In [ ]:
## Step 2 - Database Engineering
#Use Pandas to read your cleaned measurements and stations CSV data.
#Use the `engine` and connection string to create a database called `hawaii.sqlite`.
# Use `declarative_base` and create ORM classes for each table.
#You will need a class for `Measurement` and for `Station`. Make sure to define your primary keys.
#Once you have your ORM classes defined, create the tables in the database using `create_all`.


In [11]:
#import os
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float, Date
from sqlalchemy.orm import Session
from sqlalchemy.ext.automap import automap_base
import pymysql
pymysql.install_as_MySQLdb()

#!rm Hawaii_trip.sqlite



In [12]:
measurements_clean_df = pd.read_csv('clean_hawaii_measurements.csv')
measurements_clean_df.head()

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
4,USC00519397,2010-01-07,0.06,70


In [13]:
stations_clean_df = pd.read_csv('clean_hawaii_stations.csv')
stations_clean_df

,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.27160,-157.81680,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.42340,-157.80150,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.52130,-157.83740,7.0
3,USC00517948,"PEARL CITY, HI US",21.39340,-157.97510,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.49920,-158.01110,306.6
5,USC00519523,"WAIMANALO EXPERIMENTAL FARM, HI US",21.33556,-157.71139,19.5
6,USC00519281,"WAIHEE 837.5, HI US",21.45167,-157.84889,32.9
7,USC00511918,"HONOLULU OBSERVATORY 702.2, HI US",21.31520,-157.99920,0.9
8,USC00516128,"MANOA LYON ARBO 785.2, HI US",21.33310,-157.80250,152.4


In [14]:
Base = declarative_base()


In [15]:
# Create Measurments and Stations Classes
# ----------------------------------
class Measurments(Base):
    __tablename__ = 'measurment'
    id = Column(Integer, primary_key=True)
    station = Column(String)
    date = Column(Date)
    prcp = Column(Float)
    tobs = Column(Integer)


class Stations(Base):
    __tablename__ = 'station'
    id = Column(Integer, primary_key=True)
    station = Column(String)
    name = Column(String(255))
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)



In [16]:
# Right now, this table only exists in python and not in the actual database
Base.metadata.tables

immutabledict({'measurment': Table('measurment', MetaData(bind=None), Column('id', Integer(), table=<measurment>, primary_key=True, nullable=False), Column('station', String(), table=<measurment>), Column('date', Date(), table=<measurment>), Column('prcp', Float(), table=<measurment>), Column('tobs', Integer(), table=<measurment>), schema=None), 'station': Table('station', MetaData(bind=None), Column('id', Integer(), table=<station>, primary_key=True, nullable=False), Column('station', String(), table=<station>), Column('name', String(length=255), table=<station>), Column('latitude', Float(), table=<station>), Column('longitude', Float(), table=<station>), Column('elevation', Float(), table=<station>), schema=None)})

In [17]:
engine = create_engine("sqlite:///Hawaii_vacation.sqlite")
# This is where we create our tables in the database
Base.metadata.create_all(engine)
# The ORM’s “handle” to the database is the Session.

Base = automap_base()
Base.prepare(engine, reflect=True)



In [18]:
# pass the data frame to the data base
#stations_clean_df.to_sql('stations', engine, if_exists= 'replace') #, index=False)
stations_clean_df.to_sql(con=engine, index_label='id', name='station', if_exists= 'append')
#measurements_clean_df.to_sql('measurment', engine, if_exists= 'replace') #, index=False)
measurements_clean_df.to_sql(con=engine, index_label='id', name='measurment', if_exists= 'append')


In [19]:
#engine.execute('select * from Station').fetchall()

In [20]:
#engine.execute('select * from measurment').fetchall()

In [21]:
session = Session(bind=engine)
session.commit()

In [22]:
session.query(Stations.id, Stations.station, Stations.name, Stations.latitude, Stations.longitude, Stations.elevation).all()

[(0, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (1, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 (2,
  'USC00514830',
  'KUALOA RANCH HEADQUARTERS 886.9, HI US',
  21.5213,
  -157.8374,
  7.0),
 (3, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 (4, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6),
 (5,
  'USC00519523',
  'WAIMANALO EXPERIMENTAL FARM, HI US',
  21.33556,
  -157.71139,
  19.5),
 (6,
  'USC00519281',
  'WAIHEE 837.5, HI US',
  21.45167,
  -157.84888999999995,
  32.9),
 (7,
  'USC00511918',
  'HONOLULU OBSERVATORY 702.2, HI US',
  21.3152,
  -157.9992,
  0.9),
 (8, 'USC00516128', 'MANOA LYON ARBO 785.2, HI US', 21.3331, -157.8025, 152.4)]

In [23]:
session.query(Measurments.id, Measurments.station, Measurments.date, Measurments.prcp, Measurments.tobs).all()

[(0, 'USC00519397', datetime.date(2010, 1, 1), 0.08, 65),
 (1, 'USC00519397', datetime.date(2010, 1, 2), 0.0, 63),
 (2, 'USC00519397', datetime.date(2010, 1, 3), 0.0, 74),
 (3, 'USC00519397', datetime.date(2010, 1, 4), 0.0, 76),
 (4, 'USC00519397', datetime.date(2010, 1, 7), 0.06, 70),
 (5, 'USC00519397', datetime.date(2010, 1, 8), 0.0, 64),
 (6, 'USC00519397', datetime.date(2010, 1, 9), 0.0, 68),
 (7, 'USC00519397', datetime.date(2010, 1, 10), 0.0, 73),
 (8, 'USC00519397', datetime.date(2010, 1, 11), 0.01, 64),
 (9, 'USC00519397', datetime.date(2010, 1, 12), 0.0, 61),
 (10, 'USC00519397', datetime.date(2010, 1, 14), 0.0, 66),
 (11, 'USC00519397', datetime.date(2010, 1, 15), 0.0, 65),
 (12, 'USC00519397', datetime.date(2010, 1, 16), 0.0, 68),
 (13, 'USC00519397', datetime.date(2010, 1, 17), 0.0, 64),
 (14, 'USC00519397', datetime.date(2010, 1, 18), 0.0, 72),
 (15, 'USC00519397', datetime.date(2010, 1, 19), 0.0, 66),
 (16, 'USC00519397', datetime.date(2010, 1, 20), 0.0, 66),
 (17, 'USC0